In [6]:
# packages
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Simple simulations

In [7]:
# define Black Scholes model as comparison/validation
def black_scholes(S0, K, T, r, sigma):
    d1 = (np.log(S0/K) + (r + 0.5 * sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [8]:
# MC simulation for a vanilla call
def monte_carlo_european_call(S0, K, T, r, sigma, N, n_steps = 252):
    np.random.seed(42)

    # time step
    dt = T / n_steps

    # time grid
    time_grid = np.linspace(0, T, n_steps + 1)

    # store all paths
    price_paths = np.zeros((N, n_steps + 1))
    S0 = price_paths[:, 0]

    for step in range(1, n_steps + 1):
        Z = np.random.standard_normal(N)
        price_paths[:, step] = price_paths[:, step-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

    # call price
    ST = price_paths[:, -1]

    # call payoff
    payoff = np.maximum(ST - K, 0)

    # discounted price
    price = np.exp(-r * T) * np.mean(payoff)

    return price, time_grid, price_paths

In [9]:
# visualization
def mc_paths_visualization(time_grid, price_paths, S0, K, price, n_paths = 100):
    N = price_paths.shape[0]
    n_show = min(n_paths, N)
    
    # Créer la figure avec 2 graphiques
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # =========================================================================
    # GRAPHIQUE 1 : Les chemins
    # =========================================================================
    
    ax1 = axes[0]
    
    for i in range(n_show):
        ax1.plot(time_grid, price_paths[i, :], 
                alpha=0.4,           # Transparence
                linewidth=0.8,       # Épaisseur
                color='steelblue')   # Couleur
    
    # Ligne du strike
    ax1.axhline(y=K, color='red', linestyle='--', linewidth=2.5, 
               label=f'Strike K = {K}€', alpha=0.8)
    
    # Ligne du prix initial
    ax1.axhline(y=S0, color='green', linestyle='--', linewidth=2, 
               label=f'Prix initial S₀ = {S0}€', alpha=0.8)
    
    # Prix moyen final
    mean_final = np.mean(price_paths[:, -1])
    ax1.axhline(y=mean_final, color='orange', linestyle=':', linewidth=2,
               label=f'Prix moyen final = {mean_final:.2f}€', alpha=0.8)
    
    # Personnalisation
    ax1.set_xlabel('Temps (années)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Prix de l\'actif (€)', fontsize=12, fontweight='bold')
    ax1.set_title(f'Simulation Monte Carlo : {n_show} chemins\n'
                 f'Prix du call = {price:.4f}€', 
                 fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11, loc='upper left')
    ax1.grid(True, alpha=0.3, linestyle='--')
    
    # =========================================================================
    # GRAPHIQUE 2 : Distribution des prix finaux
    # =========================================================================
    
    ax2 = axes[1]
    
    # Prix finaux
    final_prices = price_paths[:, -1]
    
    # Histogramme
    ax2.hist(final_prices, bins=60, density=True, alpha=0.7,
            color='steelblue', edgecolor='black', label='Distribution simulée')
    
    # Strike
    ax2.axvline(K, color='red', linestyle='--', linewidth=2.5,
               label=f'Strike K = {K}€')
    
    # Moyenne
    ax2.axvline(mean_final, color='orange', linestyle='--', linewidth=2,
               label=f'Moyenne = {mean_final:.2f}€')
    
    # Zone dans la monnaie (in-the-money)
    ax2.axvspan(K, final_prices.max(), alpha=0.2, color='green',
               label='In-the-money')
    
    # Statistiques
    itm_pct = np.mean(final_prices > K) * 100
    
    # Personnalisation
    ax2.set_xlabel('Prix final S(T) (€)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Densité', fontsize=12, fontweight='bold')
    ax2.set_title(f'Distribution des Prix Finaux\n'
                 f'{itm_pct:.1f}% des chemins sont in-the-money', 
                 fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3, linestyle='--', axis='y')
    
    plt.tight_layout()
    plt.show()

Parameters : 
- $S_0$ : initial price (in USD)
- $K$ : strike (in USD)
- $T$ : time horizon (here, 1 year)
- $r$ : risk-neutral interest rate
- $sigma$ : volatility
- $N$ : number of simulations

In [10]:
# example with parameters
S0 = 100
K = 105
T = 1
r = 0.05
sigma = 0.2
N = 20000

In [11]:
mc_price, timegrid, pricepaths = monte_carlo_european_call(S0, K, T, r, sigma, N)
black_scholes_price = black_scholes(S0, K, T, r, sigma)

In [12]:
print(f"Monte Carlo price : {mc_price:.2f}")
print(f"Black Scholes : {black_scholes_price:.2f}")

Monte Carlo price : 0.00
Black Scholes : 8.02


In [13]:
### ADD CONFIDENCE INTERVALS

In [ ]:
# visualization
#######

# Monte Carlo simulations with variance reduction methods

## Antithetic variates

## Control variates